<a href="https://colab.research.google.com/github/ti-lei/data-course-sample/blob/main/%E3%80%8Csample_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [ ]:
# test
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-23 09:04:39--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  19.9MB/s    in 0.7s    

2021-12-23 09:04:40 (19.9 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-23 09:04:40--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.1MB/s    in 0.7s    

202

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
# 原始資料沒有 header 另外命名給他
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.to_csv('metada.csv',index=False)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
# agg(list)會把所有除了reviewerID的欄位用陣列包住
# to_dict 會把 dataframe 轉成字典(一列一個字典) 並用 陣列包住
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_trainings_by_user = ratings_trainings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
# 把 [{'asin': ['B01DKQAXC0'], 'reviewerID': 'A100XQFWKQ30O2'},..] 變成 {A100XQFWKQ30O2:['B01DKQAXC0'],...}
# key: reviwerID value: asin
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
ratings_trainings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_trainings_by_user }

# users:[user1,user2....]
users = list(ratings_testings_by_user.keys())

# 應該要測試 users 在測試的資料裡 是否有跟 訓練的資料重疊 如果沒有的話根本不知道其消費行為 根本隨便推薦:


In [ ]:
users

['A100XQFWKQ30O2',
 'A103T1QOGFCSEH',
 'A106UKKSJ2KXPF',
 'A10A7GV4D5A11V',
 'A1119JJ37ZLB8R',
 'A113UOOLBSZN52',
 'A12M4U7WK4ALCR',
 'A12T8YTW6VWT7S',
 'A1364JXGKB46MM',
 'A137DALOQFKBTI',
 'A13FEZ3WV7S2EY',
 'A13IV4I1B0RXMG',
 'A13JU88JAHN72I',
 'A13K55R6VH1OOD',
 'A13P7VFU075A',
 'A13SWYE4QLB6NG',
 'A13ZTQ0Q4ATA41',
 'A142EDN04OD62U',
 'A142I22FIC8MZK',
 'A14834QTII5TLT',
 'A14A447VPACTBC',
 'A14AP6MN5XO6LB',
 'A14CLF25IX25US',
 'A14LYXC3HTBAHI',
 'A14VUW4KZ34EOE',
 'A14Y32P26G9YL',
 'A157T25PBS7MX4',
 'A15HZDSERD85C8',
 'A15JJ8J1FGADIX',
 'A15ZCL70JXXH89',
 'A1617KN2IAWZ6J',
 'A16E0O88262HKA',
 'A16NSZ58PTVIYF',
 'A16UGDXRTDLJG5',
 'A16X9HR3UFQQXY',
 'A16Y7V1CZCWKFV',
 'A174YOBOSW9WDN',
 'A1786SKRAJXH86',
 'A17K2BUZ20WD2',
 'A17LYRFV645L0V',
 'A18LNGVXDZBTUR',
 'A19503XX7GU6J2',
 'A19HVHRZDYFEOP',
 'A19JM38B861BO4',
 'A19KGY3W1P0MHT',
 'A19KJPOF3QOJ24',
 'A19U5RT4Q8Q1T6',
 'A1AQNQUNX8B6JT',
 'A1ARU0IN6RUFFD',
 'A1BB4635KSKDMV',
 'A1BCC7X6KRTBG0',
 'A1BPJ56BXENC7',
 'A1C0RI6W65FC8B'

In [ ]:
# 將 metada 取出要用的資訊
maindata = metadata[['also_buy','brand','rank','also_view','price','asin']]
# 計算 asin 總共有多少種 確定不是唯一
maindata['asin'].value_counts()
print(len(maindata))
# 直接把資料重複的丟掉
maindata = maindata.drop_duplicates(subset=['asin'])
# 處理 Rank的部分 將 rank 轉為數字
rankseries = maindata['rank'].str.split(expand=True)
maindata['rank'] = rankseries[0]
maindata['rank'].replace(',','', regex=True, inplace=True)
maindata['rank'] = maindata['rank'].apply(pd.to_numeric,errors='coerce')

# 將產品DATA 跟　評價DATA 合併
print(len(maindata))

32892


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


32488


In [ ]:
# 把also_view 空白的資料拿掉後取出商品ID和also_view的欄位
alsoview = maindata[maindata['also_view'].astype(bool)][['also_view','asin']].to_dict('records')
# 把review ID 進行 Group 去檢查 這些人同時買了些什麼?
ratings_trainings_reviewrID_asin = ratings_trainings[['reviewerID','asin']].groupby('reviewerID').agg(list).reset_index().to_dict('records')


# 產生Asin 對應會一起買的商品

In [ ]:
# 把also_view 空白的資料拿掉後取出商品ID和also_view的欄位
alsobuy = maindata[maindata['also_buy'].astype(bool)][['also_buy','asin']].to_dict('records')
# 建立 商品 對應到會一起買的商品ID
alsobuy_by_asin = { item['asin']: item['also_buy'] for item in alsobuy }
alsobuy_by_asin

# user 傾向買銷售排名 較前面的商品(rank)
# user 的價格屬性偏向 某個價位的商品(price)
# user 之前買過的產品 發現不滿意想要找其他品牌的商品 (rating no good, also view)
# user 之前買過的商品 發現還需要其他的商品 藉由查找 alsobuy 來尋找可能會要再購買的產品 

# 銷量排名 rating (評價分數)
# rank
# 銷售時間變化 (大家最近都買這些)
# 利用品牌重複的字句去做推薦
# 


# user 之前買過的商品 獲得 比較好的評價 預期會再買一次
# users 是待推薦的測試資料集的users
def buy_it_again(users):
  for user in users:
    # 藉由從testingdata 的 user 拿到 trainingdata使用者購買的商品, 這裡可能會拿到一個陣列
    asin_ID_list = ratings_trainings_by_user[user]
    # 查找asinID 對應到的 alsobuy 的商品, asin_ID_also_buy 也可能是一個陣列
    for item in asin_ID_list:
      asin_ID_also_buy = item[alsobuy_by_asin]


In [ ]:
def buy_it_again(users):
  result = {}
  
  for user in users:
    try:
    # 藉由從testingdata 的 user 拿到 trainingdata使用者購買的商品, 這裡可能會拿到一個陣列
      if user not in ratings_testings_by_user:
        print(user,'not exist in trainings')
        asin_ID_list = ratings_testings_by_user[user]

      # 查找asinID 對應到的 alsobuy 的商品, asin_ID_also_buy 也可能是一個陣列
      for item in asin_ID_list:
        asin_ID_also_buy = item[alsobuy_by_asin]
        result[user] = asin_ID_also_buy
        print(result)
    except Exception as e:
      pass
  return result

In [ ]:
buy_it_again(users)
# ratings_testings_by_user['A100XQFWKQ30O2']

{}

## 產生推薦

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ratings_testings_by_user = training_data.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{}

## 結果評估

In [ ]:
ratings_testings_by_user

{'A100XQFWKQ30O2': ['B01DKQAXC0'],
 'A103T1QOGFCSEH': ['B01DKQAXC0'],
 'A106UKKSJ2KXPF': ['B01ENS6XDC'],
 'A10A7GV4D5A11V': ['B01BFSNF66'],
 'A1119JJ37ZLB8R': ['B01CJNZKZK'],
 'A113UOOLBSZN52': ['B01EKY0M22'],
 'A12M4U7WK4ALCR': ['B01DKQAXC0'],
 'A12T8YTW6VWT7S': ['B01G53GFKO'],
 'A1364JXGKB46MM': ['B01GLA54SA'],
 'A137DALOQFKBTI': ['B01AWXGD3M'],
 'A13FEZ3WV7S2EY': ['B01BHN3EHE'],
 'A13IV4I1B0RXMG': ['B01DKQAXC0'],
 'A13JU88JAHN72I': ['B017I6B6GK'],
 'A13K55R6VH1OOD': ['B01FNJ9MOW'],
 'A13P7VFU075A': ['B01H3ZQ2NI'],
 'A13SWYE4QLB6NG': ['B01DVLHVPQ'],
 'A13ZTQ0Q4ATA41': ['B01AE1TJV0'],
 'A142EDN04OD62U': ['B01CW24JXC'],
 'A142I22FIC8MZK': ['B01CD7JK9E'],
 'A14834QTII5TLT': ['B01GK8P0CS'],
 'A14A447VPACTBC': ['B01B3R5EFO'],
 'A14AP6MN5XO6LB': ['B01E7UKR38'],
 'A14CLF25IX25US': ['B01DLR9IDI'],
 'A14LYXC3HTBAHI': ['B01FT6TMQM'],
 'A14VUW4KZ34EOE': ['B01DKQAXC0'],
 'A14Y32P26G9YL': ['B018WCT01C'],
 'A157T25PBS7MX4': ['B01CX5KCIE'],
 'A15HZDSERD85C8': ['B01DDWA5II'],
 'A15JJ8J1FGADIX': ['B0

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 是測試資料裡 key:user value:商品ID 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

NameError: ignored